In [ ]:
from shutil import copyfile

# copy our file into the working directory (make sure it has .py suffix)
copyfile(src = "../input/supporting-files-sigma/sigma_libs.py", dst = "../working/sigma_libs.py")
copyfile(src = "../input/supporting-files-sigma/sigma_1.py", dst = "../working/sigma_1.py")
copyfile(src = "../input/supporting-files-sigma/preproc2.py", dst = "../working/preproc2.py")
# import all our functions
from sigma_libs import *
from sigma_1 import *
from preproc2 import *
tqdm.pandas() 

In [ ]:
df_market_full, df_news_full = env.get_training_data()


In [ ]:
import resource
print("Checkpoint 1")
print('Memory usage: %s (kb)' 
%resource.getrusage(resource.RUSAGE_SELF).ru_maxrss)

In [ ]:
# REDUCED
start = datetime(2016, 12, 15, 0, 0, 0).date()
df_market = df_market_full.loc[df_market_full['time'].dt.date >= start].reset_index(drop=True)
df_news = df_news_full.loc[df_news_full['time'].dt.date >= start].reset_index(drop=True)
print(df_market.shape); print(df_news.shape)
del df_market_full, df_news_full
gc.collect()

In [ ]:
# # FULL
# df_market = df_market_full
# df_news = df_news_full
# del df_market_full, df_news_full
# gc.collect()

In [ ]:
# df_news.head()
# immediate reduction
# ONLY CONCERNED WITH ASSETS IN BOTH DFRAMES AND UNIVERSE
df_m = df_market[df_market.universe == 1.0]
df_m.drop(['universe'], axis=1, inplace=True)
asset_list = set(df_m.assetName)
df_n = df_news[df_news.assetName.isin(asset_list)]
del df_market, df_news
gc.collect()

# Market data wrangling

In [ ]:
dframe = df_m
for col in dframe.columns:
    if dframe[col].isnull().sum() != 0:
        percent = (dframe[col].isnull().sum() / dframe[col].size) *100
        percent = round(percent, 4)
        print('percent null for {} = {}%'.format(col, percent))

In [ ]:
df_m.tail()

In [ ]:
df_m[df_m.assetCode == 'ZAYO.N']

In [ ]:
print(list(set(df_m.assetCode)))

# News data wrangling**

In [ ]:
df_m.shape
df_n.shape
del df_m

gc.collect()

In [ ]:
# d_type_usage(df_n)

import os, psutil  

def cpu_stats():
    pid = os.getpid()
    py = psutil.Process(pid)
    memory_use = py.memory_info()[0] / 2. ** 30
    return 'memory GB:' + str(np.round(memory_use, 2))

In [ ]:
cpu_stats()

In [ ]:
# for col in ['headlineTag', 'sourceId']:
#             df_n[col] = df_n[col].astype('category')

# drop_list = [
        
#         'firstCreated', 'sourceTimestamp',
#     ]
# df_n.drop(drop_list, axis=1, inplace=True)

In [ ]:
# df_sub = df_n[['subjects']]
# del df_n
# gc.collect()

In [ ]:
# df_sub.subjects = df_sub.subjects.progress_apply(lambda x: re.sub(r'[n{}\']', '', x)).replace(" ", "")


In [ ]:
print("Checkpoint 1")
print('Memory usage: %s (kb)' 
%resource.getrusage(resource.RUSAGE_SELF).ru_maxrss)

In [ ]:
# x = df_sub.subjects.str.split(r',', expand=True)
# x = x.progress_applymap(lambda x: x.strip() if type(x) == str else x)
# # x = x.stack()


In [ ]:
x.head()

In [ ]:
del df_sub
gc.collect()

In [ ]:
print("Checkpoint 1")
print('Memory usage: %s (kb)' 
%resource.getrusage(resource.RUSAGE_SELF).ru_maxrss)

In [ ]:
# counts = pd.DataFrame(x.values.flatten(), columns=['subs']).subs.value_counts()

In [ ]:
# list(counts.index[:50])

In [ ]:
# x.head()

In [ ]:
# df_sub = pd.get_dummies(x, prefix='aud_').groupby(level=0).sum()

In [ ]:
df_sub.head()

In [ ]:
# df_n.head()

In [ ]:
# THIS WILL ALMOST KILL THE KERNAL
df_ = preprocess2(df_n)

In [ ]:
df_m['date'] = pd.to_datetime(df_m.time).dt.date  # Add date column
# DROP TIME FEATURES FOR BOTH DFRAMES

In [ ]:
df = df_m.merge(df_, how='left', on=['assetCode', 'date', 'assetName'])

In [ ]:
del df_m
del df_
gc.collect()

In [ ]:
df = df[df.marketCommentary.notnull()]

In [ ]:
df.shape

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.shape

In [ ]:
# date = df.date
# num_target = df.returnsOpenNextMktres10.astype('float32')
# bin_target = (df.returnsOpenNextMktres10 >= 0).astype('int8')
# universe = df.universe.astype('int8')
# # Drop columns that are not features
# df.drop(['returnsOpenNextMktres10', 'date', 'universe', 'assetCode'], 
#         axis=1, inplace=True)
# df = df.astype('float32')  # Set all remaining columns to float32 datatype
# gc.collect()
# # 'returnsOpenNextMktres10', 'date', 'universe', 'assetCode', 'assetName', 'time'

Residual Return:  
Return independent of the benchmark. The residual return is the return relative to beta times the benchmark return. To be exact, an asset's residual return equals its excess return minus beta times the benchmark excess return.

In [ ]:
# train_index, test_index = train_test_split(df.index.values, test_size=0.1, 
#                                            shuffle=False)

In [ ]:
# def evaluate_model(df, target, train_index, test_index, params):
#     params['n_jobs'] = 2  # Use 2 cores/threads
#     #model = XGBClassifier(**params)
#     model = LGBMClassifier(**params)
#     model.fit(df.iloc[train_index], target.iloc[train_index])
#     return log_loss(target.iloc[test_index], model.predict_proba(df.iloc[test_index]))

In [ ]:
# param_grid = {
#     'learning_rate': [0.15, 0.1, 0.05, 0.02, 0.01],
#     'num_leaves': [i for i in range(12, 90, 6)],
#     'n_estimators': [50, 200, 400, 600, 800],
#     'min_child_samples': [i for i in range(10, 100, 10)],
#     'colsample_bytree': [0.8, 0.9, 0.95, 1],
#     'subsample': [0.8, 0.9, 0.95, 1],
#     'reg_alpha': [0.1, 0.2, 0.4, 0.6, 0.8],
#     'reg_lambda': [0.1, 0.2, 0.4, 0.6, 0.8],
# }

# best_eval_score = 0
# for i in tqdm(range(5)):  # Hundred runs
#     params = {k: np.random.choice(v) for k, v in param_grid.items()}
#     score = evaluate_model(df, bin_target, train_index, test_index, params)
#     if score < best_eval_score or best_eval_score == 0:
#         best_eval_score = score
#         best_params = params
# print("Best evaluation logloss", best_eval_score)

In [ ]:
# # Train model with full data
# clf = LGBMClassifier(**best_params)
# clf.fit(df, bin_target)

In [ ]:
# def write_submission(model, env):
# days = env.get_prediction_days()
# Generator providing test features for each prediction day.
# train dataset ends 12-31-2016

In [ ]:
#  Definitely a candidate for pipelines
# Careful - can only run once? Restart the kernal I guess..
# for (market_obs_df, news_obs_df, predictions_template_df) in days:
#     news_obs_df = preprocess_news(news_obs_df)
#     # Unstack news
#     index_df = unstack_asset_codes(news_obs_df)
#     news_unstack = merge_news_on_index(news_obs_df, index_df)
#     # Group and and get aggregations (mean)
#     news_obs_agg = group_news(news_unstack)

#     # Join market and news frames
#     market_obs_df['date'] = market_obs_df.time.dt.date
#     obs_df = market_obs_df.merge(news_obs_agg, how='left', on=['assetCode', 'date'])
#     del market_obs_df, news_obs_agg, news_obs_df, news_unstack, index_df
#     gc.collect()
#     obs_df = obs_df[obs_df.assetCode.isin(predictions_template_df.assetCode)]

#     # Drop cols that are not features
#     feats = [c for c in obs_df.columns if c not in ['date', 'assetCode', 'assetName', 'time']]

#     preds = model.predict_proba(obs_df[feats])[:, 1] * 2 - 1
#     sub = pd.DataFrame({'assetCode': obs_df['assetCode'], 'confidence': preds})
#     predictions_template_df = predictions_template_df.merge(sub, how='left').drop(
#         'confidenceValue', axis=1).fillna(0).rename(columns={'confidence':'confidenceValue'})

#     env.predict(predictions_template_df)
# #     del obs_df, predictions_template_df, preds, sub
#     gc.collect()

#     env.write_submission_file()
       
# write_submission(clf, env)